<a href="https://colab.research.google.com/github/Newball12/tuplas2/blob/main/P5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import string
import nltk
import dask.dataframe as dd

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt_tab')

# Load the data using Dask
df = dd.read_csv('tweets_dataset.csv')

# Function to clean and tokenize a tweet
def limpiar_tokenizar(texto):
    # Tokenization
    tokens = word_tokenize(texto)
    # Convert to lowercase, remove stop words, punctuation and numbers
    stop_words = set(stopwords.words('spanish'))
    tokens = [token.lower() for token in tokens if token not in stop_words and token not in string.punctuation and not token.isdigit()]
    return tokens

# Function to analyze sentiment
def analizar_sentimiento(texto):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(texto)
    return sentiment['compound']

# Apply the cleaning and tokenization function
df['tokens'] = df["texto"].apply(limpiar_tokenizar, meta=('tokens', 'object'))

# Create a new column with the tokens joined by spaces
df['texto_limpio'] = df['tokens'].apply(lambda x: ' '.join(x), meta=('texto_limpio', 'object'))

# Apply the sentiment analysis function
df['sentimiento'] = df['texto_limpio'].apply(analizar_sentimiento, meta=('sentimiento', 'float64'))

# Classify the sentiments
df['polaridad'] = df['sentimiento'].apply(lambda x: 'positivo' if x > 0.05 else ('negativo' if x < -0.05 else 'neutral'), meta=('polaridad', 'object'))

# Compute and time the execution for Dask
import time
start_time = time.time()
computed_df = df.compute()  # Trigger computation for Dask
end_time = time.time()
dask_execution_time = end_time - start_time
print(f"Dask execution time: {dask_execution_time} seconds")

# Display the first 5 results
print(computed_df.head())

# Create a DataFrame with the results of the sentiment analysis
resultados_sentimiento = computed_df['polaridad'].value_counts().reset_index()
resultados_sentimiento.columns = ['Polaridad_sentimientos', 'Conteo']

# Calculate percentages and add them to the DataFrame
resultados_sentimiento['Porcentaje'] = (resultados_sentimiento['Conteo'] / resultados_sentimiento['Conteo'].sum()) * 100
resultados_sentimiento['Porcentaje'] = resultados_sentimiento['Porcentaje'].map('{:.2f}%'.format)

# Display the table
print(resultados_sentimiento)

# Now, let's time the execution for a single-node processing using pandas
import pandas as pd

# Load the data using pandas
df_pandas = pd.read_csv('tweets_dataset.csv')

start_time = time.time()
# Apply the functions to the pandas DataFrame (same as before)
df_pandas['tokens'] = df_pandas["texto"].apply(limpiar_tokenizar)
df_pandas['texto_limpio'] = df_pandas['tokens'].apply(lambda x: ' '.join(x))
df_pandas['sentimiento'] = df_pandas['texto_limpio'].apply(analizar_sentimiento)
df_pandas['polaridad'] = df_pandas['sentimiento'].apply(lambda x: 'positivo' if x > 0.05 else ('negativo' if x < -0.05 else 'neutral'))
end_time = time.time()
pandas_execution_time = end_time - start_time
print(f"Pandas execution time: {pandas_execution_time} seconds")

print(f"Speedup with Dask: {pandas_execution_time / dask_execution_time:.2f}x")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Dask execution time: 7.3397886753082275 seconds
   id                                              texto                fecha  \
0   1       ¡Me encanta el nuevo producto! Es increíble.  2023-12-05 11:58:00   
1   2  El producto no funciona como esperaba. Muy dec...  2023-12-19 21:44:00   
2   3  El servicio al cliente fue pésimo, no volveré ...  2023-12-29 11:56:00   
3   4  ¡Es un excelente producto, lo recomiendo total...  2023-12-04 11:53:00   
4   5    Podría ser mejor, pero en general es aceptable.  2023-11-06 10:18:00   

                                              tokens  \
0  ['¡me', 'encanta', 'nuevo', 'producto', 'es', ...   
1  ['el', 'producto', 'funciona', 'esperaba', 'mu...   
2  ['el', 'servicio', 'cliente', 'pésimo', 'volve...   
3  ['¡es', 'excelente', 'producto', 'recomiendo',...   
4  ['podría', 'ser', 'mejor', 'general', 'aceptab...   

                                        texto_limpio  sentimiento polaridad  
0  [ ' ¡ m e ' ,   ' e n c a n t a ' ,   ' n u e .